# Town04 outer lane study   

In [4]:
import carla
import time
import math

def validate_figure8_waypoints(world):
    """Get and validate waypoints for figure-8 track."""
    road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    target_lane_id = -3
    waypoint_spacing = 1.0
    
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(waypoint_spacing)
    
    # Filter and sort waypoints
    road_waypoints = {}
    for road_id in road_sequence:
        road_waypoints[road_id] = [wp for wp in waypoints 
                                 if wp.road_id == road_id and wp.lane_id == target_lane_id]
        road_waypoints[road_id].sort(key=lambda x: x.s)
        
        print(f"Road {road_id}: {len(road_waypoints[road_id])} waypoints")
        
        # Validate waypoint spacing
        if len(road_waypoints[road_id]) > 1:
            first = road_waypoints[road_id][0].transform.location
            last = road_waypoints[road_id][-1].transform.location
            distance = ((last.x - first.x)**2 + 
                       (last.y - first.y)**2 + 
                       (last.z - first.z)**2)**0.5
            print(f"Road length: {distance:.1f}m")
    
    return road_waypoints

# def drive_figure_eight(world, vehicle):
#     """Drive vehicle around Town04's figure-eight."""
#     road_waypoints = validate_figure8_waypoints(world)
#     road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
#     target_speed = 30  # km/h
    
#     try:
#         while True:
#             for road_id in road_sequence:
#                 for wp in road_waypoints[road_id]:
#                     control = compute_control(vehicle, wp, target_speed)
#                     vehicle.apply_control(control)
                    
#                     if world.get_settings().synchronous_mode:
#                         world.tick()
#                     else:
#                         time.sleep(0.1)  # Adjust timing as needed
                        
#     except KeyboardInterrupt:
#         control = carla.VehicleControl(throttle=0, brake=1)
#         vehicle.apply_control(control)

# def compute_control(vehicle, target_wp, target_speed):
#     """Compute vehicle control to reach waypoint."""
#     control = carla.VehicleControl()
    
#     # Get current state
#     current_transform = vehicle.get_transform()
#     current_velocity = vehicle.get_velocity()
#     speed = 3.6 * math.sqrt(current_velocity.x**2 + current_velocity.y**2)  # km/h
    
#     # Steering based on angle difference
#     forward = current_transform.get_forward_vector()
#     right = current_transform.get_right_vector()
#     up = current_transform.get_up_vector()
#     target_vector = target_wp.transform.location - current_transform.location
    
#     forward_dot = (forward.x * target_vector.x + forward.y * target_vector.y) 
#     right_dot = (right.x * target_vector.x + right.y * target_vector.y)
    
#     steering = math.atan2(right_dot, forward_dot) / math.pi
#     control.steer = max(-1.0, min(1.0, steering))
    
#     # Speed control
#     if speed < target_speed:
#         control.throttle = 0.7
#         control.brake = 0.0
#     else:
#         control.throttle = 0.0
#         control.brake = 0.3
        
#     return control

# def setup_test_environment(world):
    # # Enable synchronous mode
    # settings = world.get_settings()
    # settings.synchronous_mode = True
    # settings.fixed_delta_seconds = 0.05
    # world.apply_settings(settings)
    
    # # Find suitable spawn point near road_id 6 (start of figure-8)
    # carla_map = world.get_map()
    # waypoint = carla_map.get_waypoint(
    #     carla_map.get_spawn_points()[0].location,
    #     project_to_road=True,
    #     lane_type=carla.LaneType.Driving
    # )
    
    # # Navigate to road_id 6, lane_id -4
    # while waypoint.road_id != 6 or waypoint.lane_id != -4:
    #     next_waypoints = waypoint.next(2.0)
    #     if not next_waypoints:
    #         raise RuntimeError("Could not find path to figure-8 road")
    #     waypoint = next_waypoints[0]
    
    # # Spawn vehicle
    # blueprint = world.get_blueprint_library().filter('vehicle.*')[0]
    # transform = waypoint.transform
    # transform.location.z += 0.5  # Prevent spawn collision
    # vehicle = world.spawn_actor(blueprint, transform)
    
    # return vehicle

## start self driving

In [5]:
# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() 

# load Town04 map
world = client.load_world('Town04')

# Initialize and validate waypoints
road_waypoints = validate_figure8_waypoints(world)


Road 6: 64 waypoints
Road length: 63.1m
Road 35: 286 waypoints
Road length: 231.0m
Road 36: 82 waypoints
Road length: 81.0m
Road 37: 41 waypoints
Road length: 37.3m
Road 38: 246 waypoints
Road length: 236.5m
Road 39: 134 waypoints
Road length: 133.0m
Road 40: 237 waypoints
Road length: 236.3m
Road 41: 210 waypoints
Road length: 194.3m
Road 42: 34 waypoints
Road length: 33.0m
Road 43: 33 waypoints
Road length: 32.2m
Road 45: 586 waypoints
Road length: 492.9m
Road 46: 139 waypoints
Road length: 141.1m
Road 47: 114 waypoints
Road length: 113.0m
Road 48: 63 waypoints
Road length: 62.0m
Road 49: 40 waypoints
Road length: 38.8m
Road 50: 178 waypoints
Road length: 155.3m


In [6]:
# import math

# def set_spectator_camera_following_car(world, vehicle):
#     spectator = world.get_spectator()
#     transform = vehicle.get_transform()
#     location = transform.location
#     rotation = transform.rotation

#     # Calculate the location 25 units behind the vehicle
#     offset_location = location - carla.Location(x=25 * math.cos(math.radians(rotation.yaw)),
#                                                 y=25 * math.sin(math.radians(rotation.yaw)))

#     # Set the spectator transform
#     spectator.set_transform(carla.Transform(offset_location + carla.Location(z=50),
#                                             carla.Rotation(pitch=-45)))
#     return spectator

import math

def set_spectator_camera_following_car(world, vehicle):
    spectator = world.get_spectator()
    transform = vehicle.get_transform()
    location = transform.location
    rotation = transform.rotation

    # Calculate the location 35 units behind the vehicle (25 + 10 units back)
    offset_location = location - carla.Location(x=15 * math.cos(math.radians(rotation.yaw)),
                                                y=55 * math.sin(math.radians(rotation.yaw)))

    # Move the camera down by 10 units
    offset_location.z += 40  # 50 units up - 10 units down

    # Set the spectator transform with the same orientation as the vehicle
    spectator.set_transform(carla.Transform(offset_location,
                                            carla.Rotation(pitch=rotation.pitch, yaw=rotation.yaw, roll=rotation.roll)))
    return spectator

def get_figure8_waypoints(world):
    #road_sequence = [6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]
    road_sequence = [35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50]
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(1.0)
    
    all_waypoints = []
    for road_id in road_sequence:
        road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == -3]
        road_waypoints.sort(key=lambda x: x.s)
        all_waypoints.extend(road_waypoints)
        
    return all_waypoints

# def draw_route(world, road_ids=[6, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50]):
def draw_route(world, road_ids=[35, 36, 37, 38, 39, 40, 41, 45, 46, 47, 48, 49, 50]):    
    colors = [
        carla.Color(255,0,0), carla.Color(0,255,0), carla.Color(0,0,255),
        carla.Color(255,255,0), carla.Color(0,255,255), carla.Color(255,0,255),
        carla.Color(128,0,0), carla.Color(0,128,0), carla.Color(0,0,128),
        carla.Color(128,128,0), carla.Color(0,128,128), carla.Color(128,0,128),
        carla.Color(192,0,0), carla.Color(0,192,0), carla.Color(0,0,192),
        carla.Color(192,192,0)
    ]
    
    for i, road_id in enumerate(road_ids):
        carla_map = world.get_map()
        waypoints = carla_map.generate_waypoints(2.0)
        road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == -3]
        road_waypoints.sort(key=lambda x: x.s)
        
        # Set spectator position
        # spectator = world.get_spectator()
        # first_wp = road_waypoints[0]
        # spectator_location = first_wp.transform.location + carla.Location(x=-20, z=30)
        # spectator_rotation = first_wp.transform.rotation
        # spectator_rotation.pitch = -20
        # spectator.set_transform(carla.Transform(spectator_location, spectator_rotation))
        
        # Draw waypoints
        for j in range(len(road_waypoints)-1):
            wp1, wp2 = road_waypoints[j], road_waypoints[j+1]
            world.debug.draw_point(wp1.transform.location, size=0.2, color=colors[i], life_time=20)
            world.debug.draw_line(wp1.transform.location, wp2.transform.location, 
                                thickness=0.1, color=colors[i], life_time=20)
        
        world.debug.draw_point(road_waypoints[-1].transform.location, 
                             size=0.2, color=colors[i], life_time=20)
        print(f"Drew route_id: {road_id}")
        time.sleep(2)

# def set_spectator_to_waypoint(world, road_id=6, lane_id=-3):
    # carla_map = world.get_map()
    # waypoints = carla_map.generate_waypoints(2.0)
    # road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == lane_id]
    # road_waypoints.sort(key=lambda x: x.s)
    
    # spectator = world.get_spectator()
    # location = road_waypoints[0].transform.location
    # spectator.set_transform(carla.Transform(location + carla.Location(z=50), carla.Rotation(pitch=-90)))        

## got to the start of a route

In [4]:
# def set_spectator_to_waypoint(world, road_id=6, lane_id=-3):
#     carla_map = world.get_map()
#     waypoints = carla_map.generate_waypoints(2.0)
#     road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == lane_id]
#     road_waypoints.sort(key=lambda x: x.s)
    
#     spectator = world.get_spectator()
#     location = road_waypoints[0].transform.location
#     rotation = road_waypoints[0].transform.rotation

#     # Calculate the location 25 units behind the vehicle
#     offset_location = location - carla.Location(x=25 * math.cos(math.radians(rotation.yaw)),
#                                                 y=25 * math.sin(math.radians(rotation.yaw)))

#     # Set the spectator transform
#     spectator.set_transform(carla.Transform(offset_location + carla.Location(z=50),
#                                             carla.Rotation(pitch=-45)))

In [6]:
# set_spectator_to_waypoint(world, road_id=6, lane_id=-3)
# vehicle.destroy()

In [7]:
import math

def set_spectator_camera_following_car(world, vehicle):
    spectator = world.get_spectator()
    transform = vehicle.get_transform()
    location = transform.location
    rotation = transform.rotation

    # Calculate the location 35 units behind the vehicle (25 + 10 units back)
    offset_location = location - carla.Location(x=35 * math.cos(math.radians(rotation.yaw)),
                                                y=35 * math.sin(math.radians(rotation.yaw)))

    # Move the camera down by 10 units
    offset_location.z += 20  # 50 units up - 10 units down

    # Set the spectator transform with the same orientation as the vehicle but looking down
    spectator.set_transform(carla.Transform(offset_location,
                                            carla.Rotation(pitch=-15, yaw=rotation.yaw, roll=rotation.roll)))
    return spectator

def test_basic_drive(world, road_id=6):
    # Setup waypoints
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(2.0)
    road_waypoints = [wp for wp in waypoints if wp.road_id == road_id and wp.lane_id == -3]
    road_waypoints.sort(key=lambda x: x.s)
    test_waypoints = get_figure8_waypoints(world)
    
    # Visualize waypoints
    for i in range(len(test_waypoints)-1):
        wp1, wp2 = test_waypoints[i], test_waypoints[i+1]
        world.debug.draw_point(wp1.transform.location, size=0.2, color=carla.Color(0, 0, 255), life_time=20)
        world.debug.draw_line(wp1.transform.location, wp2.transform.location, 
                            thickness=0.1, color=carla.Color(255, 0, 0), life_time=20)
    world.debug.draw_point(test_waypoints[-1].transform.location, size=0.2, 
                          color=carla.Color(0, 0, 255), life_time=20)
    time.sleep(1)
    
    # Spawn vehicle
    spawn_location = test_waypoints[0].transform.location
    spawn_location.z += 1
    blueprint = world.get_blueprint_library().filter('vehicle.tesla.model3')[0]
    vehicle = world.spawn_actor(blueprint, 
                              carla.Transform(spawn_location, test_waypoints[0].transform.rotation))
    
    def compute_steering(vehicle, target_wp):
        current_transform = vehicle.get_transform()
        forward = current_transform.get_forward_vector()
        target_vector = target_wp.transform.location - current_transform.location
        
        forward_dot = forward.x * target_vector.x + forward.y * target_vector.y
        right_dot = (current_transform.get_right_vector().x * target_vector.x + 
                    current_transform.get_right_vector().y * target_vector.y)
        
        return math.atan2(right_dot, forward_dot) / math.pi
    
    def distance_to_waypoint(vehicle, waypoint):
        loc = vehicle.get_location()
        wp_loc = waypoint.transform.location
        return math.sqrt((loc.x - wp_loc.x)**2 + (loc.y - wp_loc.y)**2)
    
    # Control loop
    try:
        current_target = 1  # Index of target waypoint
        while current_target < len(test_waypoints):
            target_wp = test_waypoints[current_target]
            
            # Check if reached current waypoint
            if distance_to_waypoint(vehicle, target_wp) < 2.0:
                current_target += 1
                if current_target >= len(test_waypoints):
                    break
                continue
                
            # Compute control
            steering = compute_steering(vehicle, target_wp)
            throttle = 0.3 * (1.0 - abs(steering))  # Reduce speed when turning
            
            control = carla.VehicleControl(throttle=throttle, steer=steering)
            vehicle.apply_control(control)
            
            # Debug visualization
            world.debug.draw_point(vehicle.get_location(), size=0.1, 
                                 color=carla.Color(0, 255, 0), life_time=0.1)
            set_spectator_camera_following_car(world, vehicle)
            time.sleep(0.1)
            
        # Stop vehicle
        vehicle.apply_control(carla.VehicleControl(throttle=0, brake=1.0))
        
    except KeyboardInterrupt:
        vehicle.apply_control(carla.VehicleControl(throttle=0, brake=1.0))
    
    return vehicle, test_waypoints

vehicle, waypoints = test_basic_drive(world)


## Recording data

In [9]:
import math
import os
import cv2
import numpy as np
import time
import carla
from datetime import datetime

def setup_camera(world, vehicle):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    
    camera_transform = carla.Transform(carla.Location(x=0.8, z=1.7))
    camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
    return camera

def process_image(image, output_dir, steering_angle):
    # Convert CARLA raw image to OpenCV format
    array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (image.height, image.width, 4))
    array = array[:, :, :3]  # Remove alpha channel
    
    # Save image and steering data
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    image_filename = f"{timestamp}_steering_{steering_angle:.4f}.jpg"
    cv2.imwrite(os.path.join(output_dir, image_filename), array)
    
    # Display image
    cv2.imshow('CARLA Camera', array)
    cv2.waitKey(1)

def set_spectator_camera_following_car(world, vehicle):
    spectator = world.get_spectator()
    transform = vehicle.get_transform()
    location = transform.location
    rotation = transform.rotation

    offset_location = location - carla.Location(x=35 * math.cos(math.radians(rotation.yaw)),
                                              y=35 * math.sin(math.radians(rotation.yaw)))
    offset_location.z += 20

    spectator.set_transform(carla.Transform(offset_location,
                                          carla.Rotation(pitch=-15, yaw=rotation.yaw, roll=rotation.roll)))
    return spectator

def test_basic_drive(world, road_id=6, output_dir='carla_dataset'):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Setup waypoints
    carla_map = world.get_map()
    waypoints = carla_map.generate_waypoints(2.0)
    test_waypoints = get_figure8_waypoints(world)
    
    # Visualize waypoints
    for i in range(len(test_waypoints)-1):
        wp1, wp2 = test_waypoints[i], test_waypoints[i+1]
        world.debug.draw_point(wp1.transform.location, size=0.2, color=carla.Color(0, 0, 255), life_time=20)
        world.debug.draw_line(wp1.transform.location, wp2.transform.location, 
                            thickness=0.1, color=carla.Color(255, 0, 0), life_time=20)
    world.debug.draw_point(test_waypoints[-1].transform.location, size=0.2, 
                          color=carla.Color(0, 0, 255), life_time=20)
    
    # Spawn vehicle
    spawn_location = test_waypoints[0].transform.location
    spawn_location.z += 1
    blueprint = world.get_blueprint_library().filter('vehicle.tesla.model3')[0]
    vehicle = world.spawn_actor(blueprint, 
                              carla.Transform(spawn_location, test_waypoints[0].transform.rotation))
    
    # Setup camera
    camera = setup_camera(world, vehicle)
    
    def compute_steering(vehicle, target_wp):
        current_transform = vehicle.get_transform()
        forward = current_transform.get_forward_vector()
        target_vector = target_wp.transform.location - current_transform.location
        
        forward_dot = forward.x * target_vector.x + forward.y * target_vector.y
        right_dot = (current_transform.get_right_vector().x * target_vector.x + 
                    current_transform.get_right_vector().y * target_vector.y)
        
        return math.atan2(right_dot, forward_dot) / math.pi
    
    def distance_to_waypoint(vehicle, waypoint):
        loc = vehicle.get_location()
        wp_loc = waypoint.transform.location
        return math.sqrt((loc.x - wp_loc.x)**2 + (loc.y - wp_loc.y)**2)
    
    # Setup image callback
    camera.listen(lambda image: process_image(image, output_dir, steering))
    
    # Control loop
    try:
        current_target = 1
        steering = 0.0  # Initialize steering variable
        
        while current_target < len(test_waypoints):
            target_wp = test_waypoints[current_target]
            
            if distance_to_waypoint(vehicle, target_wp) < 2.0:
                current_target += 1
                if current_target >= len(test_waypoints):
                    break
                continue
                
            steering = compute_steering(vehicle, target_wp)  # Update steering
            throttle = 0.3 * (1.0 - abs(steering))
            
            control = carla.VehicleControl(throttle=throttle, steer=steering)
            vehicle.apply_control(control)
            
            world.debug.draw_point(vehicle.get_location(), size=0.1, 
                                 color=carla.Color(0, 255, 0), life_time=0.1)
            set_spectator_camera_following_car(world, vehicle)
            time.sleep(0.1)
            
        vehicle.apply_control(carla.VehicleControl(throttle=0, brake=1.0))
        
    except KeyboardInterrupt:
        vehicle.apply_control(carla.VehicleControl(throttle=0, brake=1.0))
    
    finally:
        camera.stop()
        camera.destroy()
        cv2.destroyAllWindows()
    
    return vehicle, test_waypoints

# Usage
vehicle, waypoints = test_basic_drive(world)

In [8]:
vehicle.destroy()

True

## YouTube video
https://youtu.be/Y14ydCmd9xA